In [24]:
import spotipy
import re
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials

In [25]:
#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id='566d92e8409244689810d8207e9a07b0', client_secret='c17859621c64453e8b9d7dfc4ed14f19')
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [26]:
playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF?si=1333723a6eff4b7f"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]
print(track_uris)

['spotify:track:4LRPiXqCikLlN15c3yImP7', 'spotify:track:6Xom58OOXk2SoU711L2IXO', 'spotify:track:6Sq7ltF9Qa7SNFBsV5Cogx', 'spotify:track:3k3NWokhRRkEPhCzPmV8TW', 'spotify:track:1IHWl5LamUGEuP4ozKQSXZ', 'spotify:track:4tYFy8ALRjIZvnvSLw5lxN', 'spotify:track:1dm6z1fWB0cErMszU25dy2', 'spotify:track:0wHFktze2PHC5jDt3B17DC', 'spotify:track:41oY4WCTj5kccfesTVFnvN', 'spotify:track:5CzixCxDkRXX9mScCmah8O', 'spotify:track:5kVZxyMY1gfdCLiChsfjn1', 'spotify:track:7dSZ6zGTQx66c2GF91xCrb', 'spotify:track:02MWAaffLxlfxAUY7c5dvx', 'spotify:track:28GCbKgjlVD9eDmasGbe0T', 'spotify:track:31i56LZnwE6uSu3exoHjtB', 'spotify:track:1PckUlxKqWQs3RlWXVBLw3', 'spotify:track:1797zYiX4cKosMH836X9Gt', 'spotify:track:5Eax0qFko2dh7Rl2lYs3bx', 'spotify:track:4d4ZXH4dr5bYfgErHiZCX2', 'spotify:track:18HOjcvTVV7W8YzBuOsAPS', 'spotify:track:44XjoNvtwevktFKjvVe1vH', 'spotify:track:2uflssWlCaJ6CbTMOsUpNI', 'spotify:track:5PjdY0CKGZdEuoNab3yDmX', 'spotify:track:0E0DRHf5PfMeor0ZCwB3oT', 'spotify:track:6VrQTLzzuyGIYjUDe4kAZk',

In [27]:
def save_only_some_fields(track_response):
    #Main Artist
    artist_uri = track_response["track"]["artists"][0]["uri"]
    artist_info = sp.artist(artist_uri)
    return { 
    #Id
    'track_id' : str(track_response['track']['id']),
    
    #Track name
    'track_name' : str(track_response['track']['name']),
    

    
    #Name, popularity, genre
    'artist_names' : [artist['name'] for artist in track_response["track"]["artists"]],
    'artist_pop' : artist_info["popularity"],
    'artist_genres' : artist_info["genres"],
    
    #Album
    'album' : track_response["track"]["album"]["name"],
    
    #Popularity of the track
    'track_pop' : track_response["track"]["popularity"],

    #Duration
    'track_duration' : track_response["track"]["duration_ms"],

    'added_at': track_response['added_at']
    }

tracks = [save_only_some_fields(track) for track in sp.playlist_tracks(playlist_URI)["items"]]

In [28]:
tracks_df = pd.DataFrame(tracks)
tracks_df.head()

,track_id,track_name,artist_names,artist_pop,artist_genres,album,track_pop,track_duration,added_at
0,4LRPiXqCikLlN15c3yImP7,As It Was,[Harry Styles],90,[pop],As It Was,100,167303,2022-05-13T12:02:30Z
1,6Xom58OOXk2SoU711L2IXO,Moscow Mule,[Bad Bunny],100,"[latin, reggaeton, trap latino]",Un Verano Sin Ti,91,245939,2022-05-13T12:02:30Z
2,6Sq7ltF9Qa7SNFBsV5Cogx,Me Porto Bonito,"[Bad Bunny, Chencho Corleone]",100,"[latin, reggaeton, trap latino]",Un Verano Sin Ti,89,178567,2022-05-13T12:02:30Z
3,3k3NWokhRRkEPhCzPmV8TW,Ojitos Lindos,"[Bad Bunny, Bomba Estéreo]",100,"[latin, reggaeton, trap latino]",Un Verano Sin Ti,89,258298,2022-05-13T12:02:30Z
4,1IHWl5LamUGEuP4ozKQSXZ,Tití Me Preguntó,[Bad Bunny],100,"[latin, reggaeton, trap latino]",Un Verano Sin Ti,88,243716,2022-05-13T12:02:30Z


In [29]:
audio_features = {}

for idd in tracks_df['track_id'].tolist():
    audio_features[idd] = sp.audio_features(idd)[0]
    
tracks_df['acousticness'] = tracks_df['track_id'].apply(lambda idd: audio_features[idd]['acousticness'])
tracks_df['speechiness'] = tracks_df['track_id'].apply(lambda idd: audio_features[idd]['speechiness'])
tracks_df['key'] = tracks_df['track_id'].apply(lambda idd: str(audio_features[idd]['key']))
tracks_df['liveness'] = tracks_df['track_id'].apply(lambda idd: audio_features[idd]['liveness'])
tracks_df['instrumentalness'] = tracks_df['track_id'].apply(lambda idd: audio_features[idd]['instrumentalness'])
tracks_df['energy'] = tracks_df['track_id'].apply(lambda idd: audio_features[idd]['energy'])
tracks_df['tempo'] = tracks_df['track_id'].apply(lambda idd: audio_features[idd]['tempo'])
tracks_df['time_signature'] = tracks_df['track_id'].apply(lambda idd: audio_features[idd]['time_signature'])
tracks_df['loudness'] = tracks_df['track_id'].apply(lambda idd: audio_features[idd]['loudness'])
tracks_df['danceability'] = tracks_df['track_id'].apply(lambda idd: audio_features[idd]['danceability'])
tracks_df['valence'] = tracks_df['track_id'].apply(lambda idd: audio_features[idd]['valence'])

In [30]:
tracks_df.head()

,track_id,track_name,artist_names,artist_pop,artist_genres,album,track_pop,track_duration,added_at,acousticness,speechiness,key,liveness,instrumentalness,energy,tempo,time_signature,loudness,danceability,valence
0,4LRPiXqCikLlN15c3yImP7,As It Was,[Harry Styles],90,[pop],As It Was,100,167303,2022-05-13T12:02:30Z,0.3420,0.0557,6,0.3110,0.001010,0.731,173.930,4,-5.338,0.520,0.662
1,6Xom58OOXk2SoU711L2IXO,Moscow Mule,[Bad Bunny],100,"[latin, reggaeton, trap latino]",Un Verano Sin Ti,91,245939,2022-05-13T12:02:30Z,0.2940,0.0333,5,0.1150,0.000001,0.674,99.968,4,-5.453,0.804,0.292
2,6Sq7ltF9Qa7SNFBsV5Cogx,Me Porto Bonito,"[Bad Bunny, Chencho Corleone]",100,"[latin, reggaeton, trap latino]",Un Verano Sin Ti,89,178567,2022-05-13T12:02:30Z,0.0901,0.0817,1,0.0933,0.000027,0.712,92.005,4,-5.105,0.911,0.425
3,3k3NWokhRRkEPhCzPmV8TW,Ojitos Lindos,"[Bad Bunny, Bomba Estéreo]",100,"[latin, reggaeton, trap latino]",Un Verano Sin Ti,89,258298,2022-05-13T12:02:30Z,0.0800,0.0413,3,0.5280,0.000001,0.686,79.928,4,-5.745,0.647,0.268
4,1IHWl5LamUGEuP4ozKQSXZ,Tití Me Preguntó,[Bad Bunny],100,"[latin, reggaeton, trap latino]",Un Verano Sin Ti,88,243716,2022-05-13T12:02:30Z,0.0993,0.2530,5,0.1260,0.000291,0.715,106.672,4,-5.198,0.650,0.187


In [21]:
tracks_df['track_duration'] = tracks_df['track_duration'].apply(lambda duration: duration/1000)
tracks_df['artist_names'] = tracks_df['artist_names'].apply(lambda artists: artists[0])

In [23]:
# Output
tracks_df.to_csv("raw_data.csv")